# <font color='red'> Data collection - Part 1</font>

Enter your user name, client id, and client secret to create a read-only reddit instance.

In [51]:
import os.path

import praw

username = "LaylaTheDog1997"
clientid = "bRcb4hf7R9MnBXCl61N4NA"
clientsecret = "SM0Itf82n7e_v7wBXF7mC-ypq7YT7g"

reddit = praw.Reddit(client_id=clientid,
                     client_secret=clientsecret,
                     user_agent='data collection (by {})'.format(username),
                    redirect_uri='http://localhost:8000',)

## Retrieving and saving data - praw

We can do a search for key words in a specific subreddit or all of them.<br>

For example let's serach for the phrase *I feel guilty* in all subreddits:

In [53]:
import pandas as pd
from datetime import datetime


counter = 0
posts_table = []
column_names = ["Index", "subreddit", "post_time",
                "user_name", "title", "text",
                "num_comments", "score", "ups", "upvote_ratio", 
                "downs", 'downvote', 
                "URL",  
               "over18"]

subreddit = "AutisticAdults"
results = reddit.subreddit(subreddit).new(limit=1000)
for p in results: # p is a reddit post
    counter += 1
    # Add the post to the table
    posts_table.append([counter, p.subreddit, datetime.utcfromtimestamp(p.created),
                        p.author, p.title, p.selftext,
                        p.num_comments, p.score, p.ups, p.upvote_ratio,
                        p.downs, p.downvote,
                        p.permalink
                       , p.over_18])
    

posts_df = pd.DataFrame(posts_table, columns=column_names)

In [ ]:
posts_df.to_csv("postsASD.csv", index=False) # index=False - do not save the default index column

In [ ]:
posts_df.head()

In [ ]:
## Save all user submisions

In [ ]:
users = posts_df['user_name'].unique()
users = [u for u in users if u is not None]
submission_comments_table = []
column_names = ["Index", 'is_post',  "subreddit", "post_time",
                "user_name", "title", "text",
                "num_comments", "score", "ups", "upvote_ratio", 
                "downs", 'downvote', 'likes', 
                "URL",  "over18"]
counter = 0

for user in users:
    submissions = user.submissions.new(limit=100)
    for p in submissions:
        if p.selftext in ["", "[removed]"]:
            continue
        counter += 1
        submission_comments_table.append([counter, "yes", p.subreddit, datetime.utcfromtimestamp(p.created), 
                            p.author, p.title, p.selftext, 
                            p.num_comments, p.score, p.ups, p.upvote_ratio, 
                            p.downs, p.downvote,  p.likes,
                            p.permalink,  p.over_18])
posts_df_all = pd.DataFrame(posts_table, columns=column_names)


In [ ]:
posts_df_all.to_csv("postsASD_all.csv", index=False) # index=False - do not save the default index column

## Loading the data to Pandas and basic analysis

Now, let's see what can we learn from the data we retrieved.<br>

First, we need to open the csv file we have created.

In [ ]:
subreddits_dict = {}
subreddits = posts_df_all.subreddit.unique()
for sub in subreddits:
    users = posts_df_all[posts_df_all.subreddit == sub].user_name.unique()
    subreddits_dict[sub] = users

sorted_dict = dict(sorted(subreddits_dict.items(), key=lambda x: len(x[1]), reverse=True))

In [ ]:
for k, v in sorted_dict.items():
    if len(v) < 3:
        continue
    print(f"subreddit = {k}, #users = {len(v)}")

In [ ]:
filtered_top_subreddits = []

### Collect neurotypical posts from top subreddits

In [ ]:
import pandas as pd
from datetime import datetime


counter = 0
posts_table = []
column_names = ["Index", "subreddit", "post_time",
                "user_name", "title", "text",
                "num_comments", "score", "ups", "upvote_ratio",
                "downs", 'downvote',
                "URL",
               "over18"]

for subreddit in filtered_top_subreddits:
    results = reddit.subreddit(subreddit).new(limit=15)
    for p in results: # p is a reddit post
        if p.selftext in ["", "[removed]", "[deleted]"]:
            continue
        if p.author  in ["[deleted]"]:
            continue
        counter += 1
        # Add the post to the table
        posts_table.append([counter, p.subreddit, datetime.utcfromtimestamp(p.created),
                            p.author, p.title, p.selftext,
                            p.num_comments, p.score, p.ups, p.upvote_ratio,
                            p.downs, p.downvote,
                            p.permalink
                           , p.over_18])


posts_df_nr = pd.DataFrame(posts_table, columns=column_names)

In [ ]:
posts_df_nr.to_csv("postsNRTypical.csv", index=False) # index=False - do not save the default index column

## Matching by subreddit

In [ ]:
for subreddit in filtered_top_subreddits:
    sub_df_nr = posts_df_nr[posts_df_nr.subreddit == subreddit]
    sub_df_nr["ASD"] = False
    sub_df_asd = posts_df_all[posts_df_all.subreddit == subreddit]
    sub_df_asd["ASD"] = True
    df_both = pd.concat([sub_df_asd, sub_df_asd], axis=0)
    df_both.reset_index(inplace=True)
    df_both.to_csv(os.path.join("posts",f"{subreddit}_both.csv"), index=False)